In [65]:
import numpy as np
import os
import torch
from misc.example_helper import *
import importlib
import processing
import models
%load_ext autoreload
%autoreload 2
importlib.reload(processing)
importlib.reload(models)
from processing.parametric_data_manager import ParametricSHREDDataManager
from models.shred_models import SHRED

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import os

# Initialize ParametricSHREDDataManager
# manager = ParametricSHREDDataManager(
#     lags = 20,
#     train_size = 0.7,
#     val_size = 0.15,
#     test_size = 0.15,
#     scaling = "minmax",
#     compression = 20,
#     time=np.arange(0, 201),
#     )

# Add data to manager (with sensors)
dataset = np.load('kuramoto_sivashinsky\KuramotoSivashinsky_data.npz')
data = dataset['u'] # shape (500, 201, 100)
mu = dataset['mu'] # shape (500, 201, 2)
mobile_sensors = [
    forward_backward_walk(start=0, end = data.shape[2], timesteps=data.shape[1], forward_first=True),
    forward_backward_walk(start=0, end = data.shape[2], timesteps=data.shape[1], forward_first=False),
]

manager.add(
    data=data,
    random_sensors=2,
    stationary_sensors=[(0,), (1,)],
    # mobile_sensors=mobile_sensors,
    params=mu,
    id = 'KS'
)


compressed full_state_data: (70350, 20)
done generating dataset


In [ ]:
#seperate sensors from params, so when user enters new data, it is clear what the sensor order is and what the param order is when there are sensor data and param data from multiple fields
# do multiple fields make sense in parametric shred? 
# can different fields have different params? the result of 'mixing up' params of one field and keeping another field params constant to see effect??
manager.sensor_measurements

,trajectory,time,KS2-0,KS2-1,KS2-2,KS2-3,KS2 param 0,KS2 param 1,KS-0,KS-1,KS-2,KS-3,KS param 0,KS param 1
0,0,0,1.000000,1.093962,-0.113056,-0.747463,1.423285,1.568899,1.000000,1.093962,1.012799,1.122389,1.423285,1.568899
1,0,1,0.376289,0.546796,-0.116359,-0.219015,1.423285,1.568899,0.376289,0.546796,1.255018,1.374909,1.423285,1.568899
2,0,2,0.450057,0.619256,-0.059762,-0.145222,1.423285,1.568899,0.450057,0.619256,1.267902,1.406423,1.423285,1.568899
3,0,3,0.511200,0.682191,-0.017287,-0.089331,1.423285,1.568899,0.511200,0.682191,1.214194,1.372328,1.423285,1.568899
4,0,4,0.556852,0.728624,0.010436,-0.042774,1.423285,1.568899,0.556852,0.728624,1.106193,1.288485,1.423285,1.568899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100495,499,196,-0.658349,-0.446285,-0.332669,-1.145717,1.160304,1.782302,-0.658349,-0.446285,1.298624,1.141735,1.160304,1.782302
100496,499,197,-0.602322,-0.400786,-0.069085,-0.967043,1.160304,1.782302,-0.602322,-0.400786,0.864708,0.718628,1.160304,1.782302
100497,499,198,-0.181651,-0.137417,0.135359,-0.120816,1.160304,1.782302,-0.181651,-0.137417,0.507888,0.385593,1.160304,1.782302
100498,499,199,0.561369,0.286482,0.291639,1.265992,1.160304,1.782302,0.561369,0.286482,0.241034,0.147947,1.160304,1.782302


In [14]:
manager.input_summary

,field id,sensor id,sensor type,location/trajectory
0,KS2,KS2-0,stationary (user selected),"(0,)"
1,KS2,KS2-1,stationary (user selected),"(1,)"
2,KS2,KS2-2,stationary (user selected),"(79,)"
3,KS2,KS2-3,stationary (user selected),"(6,)"


In [15]:
# Get train/valid/test datasets
train_set, valid_set, test_set = manager.preprocess()

# Print dataset shapes
print('Data Shapes:')
print ('Reconstructor Data')
print('train X:', train_set.reconstructor.X.shape)
print('train Y:', train_set.reconstructor.Y.shape)
print('valid X:', valid_set.reconstructor.X.shape)
print('valid Y:', valid_set.reconstructor.Y.shape)
print('test X:', test_set.reconstructor.X.shape)
print('test Y:', test_set.reconstructor.Y.shape)

Data Shapes:
Reconstructor Data
train X: torch.Size([70350, 21, 6])
train Y: torch.Size([70350, 20])
valid X: torch.Size([15075, 21, 6])
valid Y: torch.Size([15075, 20])
test X: torch.Size([15075, 21, 6])
test Y: torch.Size([15075, 20])


In [62]:
result = manager.postprocess(train_set.reconstructor.Y)

field_spatial_dim 20
field_data.shape (70350, 20)


In [36]:
r = result['KS']

In [ ]:
r.shape # further reshape

(70350, 100)

In [27]:



# shape (500, 201, 100)

In [ ]:
# initialize SHRED
shred = SHRED(sequence='LSTM', decoder='SDN')
# fit SHRED
shred.fit(train_set, valid_set, num_epochs=10)

Epoch 2/10:  11%|█▏        | 126/1100 [00:14<00:10, 91.15batch/s, loss=0.00377, L2=0.133]

In [ ]:
prediction = shred._reconstructor(test_set.reconstructor.X).detach().cpu().numpy()
prediction.shape

In [ ]:
### given sensor measurements of shape (ntraj, ntimes, nsensors) -> (ntraj*ntimes, lags, nsensors) and params of (ntraj, ntimes, nparams) -> (ntaj*ntimes, lags, nsensors)


In [ ]:
# development testing
train_set.reconstructor.X[201,:,0]

In [ ]:
# NEW DATA
# n_traj, n_times, nsensors
# n_traj, n_times, nparams


